In [1]:
import torch
import time
import gc
import os
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from math import ceil
from copy import deepcopy 
from tabulate import tabulate
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
from collections import namedtuple
from collections import defaultdict as dd

In [2]:
### Basic structure

DATA_DIR = "data/"
STOCKFISH_DIR = 'stockfish/'
ARCHIVE_DIR = DATA_DIR + 'archives/'

###
##
###

### Stockfish

STOCKFISH_AVX512_TAR = "stockfish-ubuntu-x86-64-avx512.tar"
STOCKFISH_AVX512 = "stockfish-ubuntu-x86-64-avx512"
STOCKFISH_AVX512_EXE = STOCKFISH_DIR + STOCKFISH_AVX512


###
##
###

### URLs

ELITE_DATABASE_URL  = "https://database.nikonoel.fr/lichess_elite_2021-11.zip"
STOCKFISH_DOWNSTREAM = "https://github.com/official-stockfish/Stockfish/releases/latest/download/"
STOCKFISH_AVX512_URL = STOCKFISH_DOWNSTREAM + STOCKFISH_AVX512_TAR

### 
##
###

### Datasets 

ELITE_DATASET_ARCHIVE = "lichess_elite_2021-11.zip"
ELITE_DATASET_FILENAME = "lichess_elite_2021-11.pgn"


LICHESS_EVAL_ARCHIVE = ARCHIVE_DIR + "lichess_db_eval.jsonl.zst"
LICHESS_EVAL_FILENAME = DATA_DIR + "lichess_db_eval.jsonl"


In [3]:
BITBOARD_DIR = DATA_DIR + 'bitboards/'
ELITE_DATA_BASE_URL  = "https://database.nikonoel.fr/"
STOCKFISH_DOWNSTREAM = "https://github.com/official-stockfish/Stockfish/releases/latest/download/"

SAMPLE_ZIP = "lichess_elite_2021-11.zip"
SAMPLE_PGN = "lichess_elite_2021-11.pgn"
SAMPLE_BITBOARD = "elite_bitboard.csv"
BITBOARD_1M = "1M.csv"
BITBOARD_10M = "10M.csv"
ELITE_DATA_SAMPLE_URL = ELITE_DATA_BASE_URL + SAMPLE_ZIP
SAMPLE_ZIP_FILE = ARCHIVE_DIR + SAMPLE_ZIP
SAMPLE_PGN_FILE  = DATA_DIR + SAMPLE_PGN

SAMPLE_BITBOARD_FILE = BITBOARD_DIR + SAMPLE_BITBOARD
BITBOARD_10M_FILE = BITBOARD_DIR + BITBOARD_10M
BITBOARD_1M_FILE = BITBOARD_DIR + BITBOARD_1M

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

# Dataset and models

In [4]:
# def sizeof_fmt(num, suffix="B"):
#     for unit in ("", "Ki", "Mi", "Gi"):
#         if abs(num) < 1024.0:
#             return f"{num:3.1f}{unit}{suffix}"
#         num /= 1024.0
#     return f"{num:.1f}Yi{suffix}"
    
def sizeof_fmt(num):
    for unit in ("", "K", "M", "G"):
        if abs(num) < 1000.0:
            return f"{num:.0f}{unit}"
        num /= 1000.0
    return f"{num:.0f}"


print(f"Allocated: {sizeof_fmt(torch.cuda.memory_allocated())}") 
print(f"Reserved: {sizeof_fmt(torch.cuda.memory_reserved())}") 

Allocated: 0
Reserved: 0


In [5]:
class BitboardDrawDatasetSimple(Dataset):
    def __init__(self, bitboard_file):
        bitboards_df = pd.read_csv(bitboard_file, dtype="uint64", usecols=range(12))
        metadata_df = pd.read_csv(bitboard_file)

        self.bitboards = self.bitboards_to_layers(bitboards_df)
        # self.meta_features = self.binary_features_to_layers(metadata_df[["white", "cK", "cQ", "ck", "cq"]])
        self.is_draw = metadata_df['draw'].to_numpy(dtype=np.single)
        self.length = self.is_draw.size

        # self.bitboards = np.hstack((self.bitboards, self.meta_features))

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        return self.bitboards[idx], self.is_draw[idx]

    def bitboards_to_layers(self, bitboards):
        cont = np.ascontiguousarray(np.expand_dims(bitboards.to_numpy(), 2)).view(np.uint8)
        return np.unpackbits(np.flip(cont, axis=2), axis=2).astype(np.single).reshape(-1, 768)

    def binary_features_to_layers(self, features):
        i = (features.to_numpy(dtype=np.uint64) - 1) ^ 0xffffffffffffffff
        cont = np.ascontiguousarray(np.expand_dims(i, 2)).view(np.uint8)
        return np.unpackbits(np.flip(cont, axis=2), axis=2).astype(np.single).reshape(-1, 768)

In [21]:
class BitboardDrawDataset(Dataset):

    @classmethod
    def from_dataset_info(cls, dataset_info):
        return cls(dataset_info.source_file, dataset_info.chunk_size, dataset_info.shuffle, dataset_info.in_memory)
    
    def __init__(self, bitboard_file, chunk_size, shuffle = True, in_memory = False):
        self.curr_batch = 0
        self.shuffle = shuffle
        self.bitboard_file = bitboard_file
        self.chunk_size = chunk_size
        self.datasamples = self.calculate_dataset_size()
        
        self.batches = ceil(self.datasamples / chunk_size)
        self.splits = np.arange(chunk_size, self.datasamples, chunk_size)
        self.indices = np.arange(1, self.datasamples+1)
        self.splits = np.append(self.splits, [0])

        self.in_memory = in_memory or self.chunk_size == self.datasamples
        
        self.load_data()
    
    def __len__(self):
        return self.datasamples
        
    def __getitem__(self, idx):
        batch = idx // self.chunk_size
        # Laod new data 
        if self.curr_batch != batch and not self.in_memory:
            self.curr_batch = batch
            self.load_data()            

        idx -= self.chunk_size * self.curr_batch
        
        return self.bitboards[idx], self.is_draw[idx]

    def bitboards_to_layers(self, bitboards):
        cont = np.ascontiguousarray(np.expand_dims(bitboards.to_numpy(), 2)).view(np.uint8)
        return np.unpackbits(np.flip(cont, axis=2), axis=2).astype(np.single).reshape(-1, 768)

    def binary_features_to_layers(self, features):
        i = (features.to_numpy(dtype=np.uint64) - 1) ^ 0xffffffffffffffff
        cont = np.ascontiguousarray(np.expand_dims(i, 2)).view(np.uint8)
        return np.unpackbits(np.flip(cont, axis=2), axis=2).astype(np.single).reshape(-1, 768)
        
    def load_data(self):
        if self.curr_batch == 0 and self.shuffle:
            np.random.shuffle(self.indices)
            
        ignore = set(self.indices)

        if self.curr_batch == self.batches-1:
            ignore.difference_update(self.indices[self.splits[self.curr_batch-1]:])
        else:
            ignore.difference_update(self.indices[self.splits[self.curr_batch-1]:self.splits[self.curr_batch]])

        df = pd.read_csv(self.bitboard_file, dtype="uint64", usecols=range(18), skiprows=ignore)
        self.bitboards = self.bitboards_to_layers(df.iloc[:, range(12)])
        # self.meta_features = df[["white", "cK", "cQ", "ck", "cq"]].to_numpy(dtype=np.float32)
        self.is_draw = df['draw'].to_numpy(dtype=np.single)

        # self.bitboards = np.hstack((self.bitboards, self.meta_features))
    
    def calculate_dataset_size(self):
        with open(self.bitboard_file) as f:
            return sum(1 for line in f) - 1

    def dataloader(self, batch_size):
        return DataLoader(self, batch_size=batch_size, shuffle=self.in_memory)

DatasetInfo = namedtuple("DatasetInfo", ["source_file", "chunk_size", "shuffle", "in_memory"])

In [48]:
class DenseNetwork(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.sizes = [config['input_size']] + config['hidden_sizes'] + [1]
        normalization = config.get('normalization', False)
        dropout = config.get('dropout', False)
        self.layers = len(self.sizes) - 2
        ll = []

        for i in range(self.layers+1):
            ll.append(nn.Linear(self.sizes[i], self.sizes[i+1]))   
            if i < self.layers:
                if self.layers > 1 and normalization:
                    ll.append(nn.BatchNorm1d(self.sizes[i+1]))
                if self.layers > 0 and dropout:
                    ll.append(nn.Dropout(0.5))
                ll.append(nn.ReLU())
                
        self.model = nn.Sequential(*ll)

    def forward(self, x):
        return self.model.forward(x)

    def __str__(self):
        return f"Dense_{'_'.join(map(str, self.sizes))}"

    def get_hidden_layer_count(self):
        return self.layers

In [65]:
class ConvLayer(nn.Module):

    def __init__(self, in_layers, out_layers, ksize=3, stride=1, padding=1):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_layers, out_layers, kernel_size=ksize, stride=stride, padding=padding),
            # nn.BatchNorm2d(out_layers),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.model.forward(x)

In [64]:
class ResLayer(nn.Module):

    def __init__(self, in_layers, out_layers, ksize=3, stride=1, padding=1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_layers, out_layers, kernel_size=ksize, stride=stride, padding=padding),
            nn.BatchNorm2d(out_layers),
            nn.ReLU(),
            nn.Conv2d(out_layers, out_layers, kernel_size=ksize, stride=stride, padding=padding),
            nn.BatchNorm2d(out_layers),
        )

    def forward(self, x):
        return nn.functional.relu(self.conv(x) + x)

In [75]:
class ConvolutionNetwork(nn.Module):

    def __init__(self, neurons=256):
        super().__init__()
        self.neurons = neurons
        self.model = nn.Sequential(
            nn.Unflatten(1, (12, 8, 8)),
            ConvLayer(12, 32),
            # nn.MaxPool2d(kernel_size=3, stride=1),

            ConvLayer(32, 64),
            nn.MaxPool2d(kernel_size=2, stride=2),

            ConvLayer(64, 128),
            # nn.AvgPool2d(kernel_size=2, stride=2),

            ConvLayer(128, 256),
            # nn.AvgPool2d(kernel_size=2, stride=2),
            
            nn.Flatten(1),
            DenseNetwork({'input_size': 256 * 4 * 4,
                          'hidden_sizes': [1024, 256],
                          'normalization': False})
        )

    def forward(self, x):
        return self.model.forward(x)

    def get_hidden_layer_count(self):
        return 'conv'

    def __str__(self):
        return f"ConvNet"

In [11]:
class MiniAlphaZeroNetwork(nn.Module):

    def __init__(self, res_layers, layers):
        super().__init__()
        ll = []

        for i in range(res_layers):
            ll.append(ResLayer(layers, layers))

        self.model = nn.Sequential(
            nn.Unflatten(1, (12, 8, 8)),
            ConvLayer(12, layers),
            *ll,
            nn.Flatten(1),
            DenseNetwork({'input_size': layers*8*8,
                          'hidden_sizes': [1024, 256, 128],
                          'normalization': False})
        )

    def forward(self, x):
        return self.model.forward(x)

    def get_hidden_layer_count(self):
        return 3

    def __str__(self):
        return f"MiniAlphaZero"

# Training

In [58]:
def timeit(f):

    def timed(*args, **kw):

        ts = time.time()
        result = f(*args, **kw)
        te = time.time()

        print(f"Took: {te-ts:.2f}s")
        return result
    return timed
    

class Train:
    
    def __init__(self, train_dataset, validate_dataset, batch_size):
        self.batch_size = batch_size
        
        self.train_dataset = train_dataset
        self.validate_dataset = validate_dataset
        
        self.train_dataloader = DataLoader(self.train_dataset, batch_size=batch_size, shuffle=True)
        self.validate_dataloader = DataLoader(self.validate_dataset, batch_size=batch_size, shuffle=True)

        self.loss_fn = nn.BCEWithLogitsLoss()
        self.total_batches = len(self.train_dataloader)
        self.epoch_print_interval = 1
        
    def train_one_epoch(self, model, optimizer):
        model.train(True)
        for i, data in enumerate(self.train_dataloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.unsqueeze(1).to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = self.loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
    
    @timeit
    def train(self, model, optimizer, epochs, stop_early=10, p=True):
        best_result = dd(None)
        history = dd(lambda: np.zeros(epochs))
        downward = 0
        try:
            best_result['vloss'] = np.inf
        
            for epoch in range(1, epochs + 1):
                if p and epoch % self.epoch_print_interval == 0 or epoch == 1: print(f'EPOCH {epoch}')
            
                self.train_one_epoch(model, optimizer)
            
                train_acc, train_loss = self.test(model, self.train_dataloader)
                validate_acc, validate_loss = self.test(model, self.validate_dataloader)

                if p and (epoch % self.epoch_print_interval == 0 or epoch == 1): 
                    self.print_results([train_loss, validate_loss], [train_acc, validate_acc], ["Train", "Validate"])
                        
                train_loss = train_loss.cpu().numpy()
                train_acc = train_acc.cpu().numpy()
                validate_loss = validate_loss.cpu().numpy()
                validate_acc = validate_acc.cpu().numpy()
                
                history['tloss'][epoch-1] = train_loss
                history['tacc'][epoch-1] = train_acc
                history['vloss'][epoch-1] = validate_loss
                history['vacc'][epoch-1] = validate_acc

                if validate_loss < best_result['vloss']:
                    best_result['vloss'] = validate_loss
                    best_result['model'] = deepcopy(model.state_dict())
                    best_result['epoch'] = epoch
                    best_result['acc']   = validate_acc
                    downward = 0
                else:
                    downward += 1
                    if downward >= stop_early:
                        break
                    
        except KeyboardInterrupt:
            self.save_model(model, best_result['model'], best_result['epoch'], optimizer.param_groups[0]['lr'], optimizer.param_groups[0]['momentum'], best_result['acc'])

        return best_result, history

    def print_results(self, loss, acc, headers):
        acc = list(map(lambda x: f"{x:.2f}%", acc))
        print(tabulate([["Loss"] + loss,
                        ["Accuracy"] + acc],
                       headers=[""] + headers))

    def test(self, model, dataloader):
        acc, loss = 0, 0
        model.eval()
        with torch.no_grad():
            for i, (vinputs, vlabels) in enumerate(dataloader):
                vinputs, vlabels = vinputs.to(device), vlabels.unsqueeze(1).to(device)
                voutputs = model(vinputs)
                pred = nn.functional.sigmoid(voutputs).round()
                
                acc += (pred == vlabels).sum() / self.batch_size
                loss += self.loss_fn(voutputs, vlabels) 
            
            acc = acc / (i+1) * 100
            loss /= (i+1)
        
        return acc, loss
    
    def find_best(self, model, epochs, lr=1e-3, momentum=0.9, lr_iter=3, lr_step=0.5):
        print(str(model))
        
        for i in range(lr_iter):
            print(f"Staring with lr {lr}")
            model.to(device)
            optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=0.001)
            best_results, hist = self.train(model, optimizer, epochs)
            lr *= lr_step
            model.to('cpu')
            model.load_state_dict(best_results["model"])
            
        self.save_model(model, best_results['model'], best_results['epoch'], lr, momentum, best_results['acc'])
        self.cleanup()
        
        print("==== BEST RESULTS ====")
        print(f"EPOCH: {best_results["epoch"]}")
        self.print_results([best_results['vloss']], [best_results['acc']], ["best_model"])
        print()
        
        return best_results, hist

    def save_model(self, model, state_dict, epoch, lr, momentum, acc):
        base_path = f"models/{sizeof_fmt(len(self.train_dataset))}/{str(model.get_hidden_layer_count())}l/"
        if(not os.path.isdir(base_path)):
            !mkdir -p {base_path}
        
        model_path = base_path + f"{str(model)}_b{self.batch_size}_e{epoch}_lr{lr}_m{momentum}_acc{acc:.2f}"
        torch.save(state_dict, model_path)
            
    def cleanup(self):  
        gc.collect()
        torch.cuda.empty_cache()
        

In [13]:
def test_model(model, dataset_info, batch_size=512, loss_fn=nn.BCEWithLogitsLoss()):
    model.to(device)
    dataloader = BitboardDrawDataset.from_dataset_info(dataset_info).dataloader(batch_size)
    acc, loss = 0, 0
    model.eval()
    with torch.no_grad():
        for i, (vinputs, vlabels) in enumerate(dataloader):
            vinputs, vlabels = vinputs.to(device), vlabels.unsqueeze(1).to(device)
            voutputs = model(vinputs)
            pred = nn.functional.sigmoid(voutputs).round()

            acc += (pred == vlabels).sum() / batch_size
            loss += loss_fn(voutputs, vlabels)

        acc = acc / (i+1) * 100
        loss /= (i+1)

    return acc, loss

In [14]:
def standard_training(train_dataset_info, test_dataset_info, model, batch_size=512, epochs=50, lr=1e-2, momentum=0.9, lr_iter=3, lr_step=0.5):
    dataset = BitboardDrawDataset.from_dataset_info(train_dataset_info)
    train, validation = random_split(dataset, [0.9, 0.1])
    
    trainer = Train(train, validation, batch_size)

    best_result, history = trainer.find_best(model, epochs, lr=lr, momentum=momentum, lr_iter=lr_iter, lr_step=lr_step)
    model.load_state_dict(best_result["model"])
    model.to(device)
    test = BitboardDrawDataset.from_dataset_info(test_dataset_info).dataloader(batch_size)
    train = DataLoader(train, batch_size=batch_size, shuffle=True)
    validate = DataLoader(validation, batch_size=batch_size, shuffle=True)
    print("==== TESTING BEST MODEL ====")

    acc, loss   = trainer.test(model, test)
    tacc, tloss = trainer.test(model, train)
    vacc, vloss = trainer.test(model, validate)

    trainer.print_results([tloss, vloss, loss], [tacc, vacc, acc], ["Train", "Validate", "Test"])
    
    return best_result, history 

In [76]:
eval_dataset_info_3k = DatasetInfo("data/eval_dataset/bitboards/training/3K_set1_train_3000.csv", 3000, False, True)
eval_dataset_info_300k = DatasetInfo("data/eval_dataset/bitboards/training/300K_set1_train_300000.csv", 300000, False, True)
eval_dataset_info_6M = DatasetInfo("data/eval_dataset/bitboards/training/6M_set1_train_6000000.csv", 6000000, False, True)

eval_test_dataset_info_3k = DatasetInfo("data/eval_dataset/bitboards/testing/3K_set1_test_300.csv", 300, False, True)
eval_test_dataset_info_300k = DatasetInfo("data/eval_dataset/bitboards/testing/300K_set1_test_30000.csv", 30000, False, True)
eval_test_dataset_info_6M = DatasetInfo("data/eval_dataset/bitboards/testing/6M_set1_test_600000.csv", 600000, False, True)

# validate_dataset_info = DatasetInfo("data/eval_dataset/bitboards/1000000_30000000.csv", 1000000, False, True)

In [78]:
sum([i[1].shape.numel() for i in list(model.model.named_parameters())])

4848993

In [77]:
model

ConvolutionNetwork(
  (model): Sequential(
    (0): Unflatten(dim=1, unflattened_size=(12, 8, 8))
    (1): ConvLayer(
      (model): Sequential(
        (0): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
      )
    )
    (2): ConvLayer(
      (model): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
      )
    )
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): ConvLayer(
      (model): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
      )
    )
    (5): ConvLayer(
      (model): Sequential(
        (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
      )
    )
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): DenseNetwork(
      (model): Sequential(
        (0): Linear(in_features=4096, out_features=1024, bias=True)
        (1): Re

In [72]:
# model = DenseNetwork(3, [768, 10_000, 256, 128, 1])
# model = DenseNetwork({'input_size': 768, 'hidden_sizes': [768, 512, 256, 128, 64], 'normalization': False})
# model = DenseNetwork({'input_size': 768, 'hidden_sizes': [768, 256, 128], 'normalization': False})
# model = DenseNetwork({'input_size': 768, 'hidden_sizes': [768, 256], 'normalization': False})
# model = DenseNetwork({'input_size': 768, 'hidden_sizes': [768], 'normalization': False})
# model = DenseNetwork({'input_size': 768, 'hidden_sizes': [], 'normalization': False, 'dropout': True})
model = ConvolutionNetwork()
# model = MiniAlphaZeroNetwork(6, 32)
# model.load_state_dict(torch.load("models/270K/0l/Dense_768_1_b512_e3_lr0.00125_m0.9_acc62.43", map_location="cpu"))

In [64]:
test_model(model, eval_test_dataset_info_300k)

(tensor(62.9436, device='cuda:0'), tensor(0.6318, device='cuda:0'))

In [74]:
histories = []
hidden_sizes = [[], [768], [768, 256], [768, 256, 128]]

for sizes in hidden_sizes:
    size = len(hidden_sizes)
    results = {"size": size, "histories": []}
    
    model = (
        DenseNetwork({'input_size': 768, 'hidden_sizes': sizes, 'normalization': False}))
    _, history = standard_training(eval_dataset_info_3k, eval_test_dataset_info_3k, model, epochs=50, lr=1e-2, lr_iter=3)
    results["histories"].append(history)

    model = DenseNetwork({'input_size': 768, 'hidden_sizes': sizes, 'normalization': False})    
    _, history = standard_training(eval_dataset_info_300k, eval_test_dataset_info_300k, model, epochs=50, lr=1e-2, lr_iter=3)
    results["histories"].append(history)
    
    model = DenseNetwork({'input_size': 768, 'hidden_sizes': sizes, 'normalization': False})    
    _, history = standard_training(eval_dataset_info_6M, eval_test_dataset_info_6M, model, epochs=50, lr=1e-2, lr_iter=3)
    results["histories"].append(history)

NameError: name 'histories' is not defined

In [79]:
_, history = standard_training(eval_dataset_info_6M, eval_test_dataset_info_6M, model, epochs=50, lr=1e-2, lr_iter=3)

ConvNet
Staring with lr 0.01
EPOCH 1
          Train               Validate
--------  ------------------  ------------------
Loss      0.6113823056221008  0.6115251183509827
Accuracy  64.58%              64.52%
EPOCH 2
          Train               Validate
--------  ------------------  -----------------
Loss      0.6098946928977966  0.610159695148468
Accuracy  64.81%              64.70%
EPOCH 3
          Train               Validate
--------  ------------------  ------------------
Loss      0.6122629046440125  0.6125297546386719
Accuracy  64.91%              64.89%
EPOCH 4
          Train               Validate
--------  ------------------  ------------------
Loss      0.6082699298858643  0.6086708307266235
Accuracy  65.02%              64.86%
EPOCH 5
          Train               Validate
--------  ------------------  ------------------
Loss      0.6044137477874756  0.6049788594245911
Accuracy  65.41%              65.30%
EPOCH 6
          Train               Validate
--------  ------

In [165]:
histories = []
hidden_sizes = [[], [768], [768, 256], [768, 256, 128]]

for sizes in hidden_sizes:
    size = len(hidden_sizes)
    results = {"size": size, "histories": []}
    
    model = (
        DenseNetwork({'input_size': 768, 'hidden_sizes': sizes, 'normalization': False}))
    _, history = standard_training(eval_dataset_info_3k, eval_test_dataset_info_3k, model, epochs=50, lr=1e-2, lr_iter=3)
    results["histories"].append(history)

    model = DenseNetwork({'input_size': 768, 'hidden_sizes': sizes, 'normalization': False})    
    _, history = standard_training(eval_dataset_info_300k, eval_test_dataset_info_300k, model, epochs=50, lr=1e-2, lr_iter=3)
    results["histories"].append(history)
    
    model = DenseNetwork({'input_size': 768, 'hidden_sizes': sizes, 'normalization': False})    
    _, history = standard_training(eval_dataset_info_6M, eval_test_dataset_info_6M, model, epochs=50, lr=1e-2, lr_iter=3)
    results["histories"].append(history)

Dense_768_1
Staring with lr 0.01
EPOCH 1
          Train              Validate
--------  -----------------  ------------------
Loss      0.688189685344696  0.6902273893356323
Accuracy  49.51%             33.79%
EPOCH 2
          Train               Validate
--------  ------------------  ------------------
Loss      0.6807668209075928  0.6886028051376343
Accuracy  51.73%              34.18%
EPOCH 3
          Train               Validate
--------  ------------------  ------------------
Loss      0.6803851127624512  0.6873668432235718
Accuracy  53.16%              34.38%
EPOCH 4
          Train               Validate
--------  ------------------  ------------------
Loss      0.6810547709465027  0.6833670735359192
Accuracy  54.30%              35.16%
EPOCH 5
          Train               Validate
--------  ------------------  ------------------
Loss      0.6779109835624695  0.6792144179344177
Accuracy  54.65%              37.30%
EPOCH 6
          Train               Validate
--------  ----

In [66]:
results = standard_training(eval_dataset_info_300k, eval_test_dataset_info_300k, model, epochs=50, lr=1e-2, lr_iter=3, lr_step=0.6)

Dense_768_768_512_256_128_64_1
Staring with lr 0.01
EPOCH 1
          Train               Validate
--------  ------------------  ------------------
Loss      0.6925826072692871  0.6923688054084778
Accuracy  50.83%              50.97%
EPOCH 2
          Train               Validate
--------  ------------------  ------------------
Loss      0.6913517713546753  0.6910330653190613
Accuracy  50.83%              50.97%
EPOCH 3
          Train               Validate
--------  ------------------  ------------------
Loss      0.6636525392532349  0.6627515554428101
Accuracy  61.07%              60.93%
EPOCH 4
          Train               Validate
--------  ------------------  ------------------
Loss      0.6333649754524231  0.6332727670669556
Accuracy  62.64%              62.07%
EPOCH 5
          Train               Validate
--------  ------------------  ----------------
Loss      0.6268425583839417  0.62774658203125
Accuracy  63.52%              62.77%
EPOCH 6
          Train               Vali

In [163]:
results = standard_training(eval_dataset_info_6M, eval_test_dataset_info_6M, model, epochs=50, lr=1e-2, lr_iter=3, lr_step=0.6)

Dense_768_768_512_256_128_64_1
Staring with lr 0.01
EPOCH 1
          Train               Validate
--------  ------------------  ------------------
Loss      0.6003141403198242  0.6023347973823547
Accuracy  66.49%              66.20%
EPOCH 2
          Train               Validate
--------  ------------------  ------------------
Loss      0.5781807899475098  0.5813451409339905
Accuracy  68.08%              67.72%
EPOCH 3
          Train               Validate
--------  ------------------  ------------------
Loss      0.5819671750068665  0.5848686099052429
Accuracy  67.52%              67.25%
EPOCH 4
          Train               Validate
--------  ------------------  ------------------
Loss      0.5632081031799316  0.5666087865829468
Accuracy  69.24%              68.83%
EPOCH 5
          Train               Validate
--------  ------------------  ------------------
Loss      0.5668482780456543  0.5707990527153015
Accuracy  68.83%              68.41%
EPOCH 6
          Train              V

In [19]:
results = standard_training(eval_dataset_info_3k, eval_test_dataset_info_3k, model, epochs=50, lr=1e-2, lr_iter=3, lr_step=0.6)

Dense_768_768_512_256_128_64_1
Staring with lr 0.01
EPOCH 1
          Train              Validate
--------  -----------------  ------------------
Loss      0.693126916885376  0.6929090619087219
Accuracy  44.04%             32.42%
EPOCH 2
          Train               Validate
--------  ------------------  ------------------
Loss      0.6931037902832031  0.6930581331253052
Accuracy  48.57%              34.38%
EPOCH 3
          Train               Validate
--------  ------------------  ------------------
Loss      0.6931018233299255  0.6932237148284912
Accuracy  43.91%              26.17%
EPOCH 4
          Train               Validate
--------  ------------------  ------------------
Loss      0.6930736303329468  0.6932661533355713
Accuracy  43.85%              26.17%
EPOCH 5
          Train               Validate
--------  ------------------  ------------------
Loss      0.6931087374687195  0.6933226585388184
Accuracy  43.85%              26.17%
EPOCH 6
          Train               Vali

In [177]:
# model = ConvolutionNetwork()
model = MiniAlphaZeroNetwork(10, 256)

In [179]:
results = standard_training(eval_dataset_info_300k, eval_test_dataset_info_300k, model, epochs=50, lr=1e-2, lr_iter=3, lr_step=0.6)

MiniAlphaZero
Staring with lr 0.01
EPOCH 1
          Train               Validate
--------  ------------------  ------------------
Loss      0.6150579452514648  0.6260372996330261
Accuracy  64.86%              63.71%
EPOCH 2
          Train               Validate
--------  ------------------  ------------------
Loss      0.5831732153892517  0.6044223308563232
Accuracy  67.67%              65.10%
EPOCH 3
          Train               Validate
--------  ------------------  ------------------
Loss      0.5792233943939209  0.6180903911590576
Accuracy  67.47%              62.02%
EPOCH 4
          Train               Validate
--------  ------------------  ------------------
Loss      0.5452953577041626  0.6081230044364929
Accuracy  70.69%              65.36%
EPOCH 5
          Train               Validate
--------  ------------------  ------------------
Loss      0.5083345770835876  0.5975314378738403
Accuracy  75.81%              65.49%
EPOCH 6
          Train               Validate
--------

## Results of training

In [24]:
def save_weights_to_csv(model: nn.Module, directory: str):
    if not os.path.exists(directory):
        os.makedirs(directory)

    for name, param in model.named_parameters():
        layer_name, param_type = name.rsplit('.', 1)
        param_data = param.detach().cpu().numpy()
        df = pd.DataFrame(param_data)
        filename = f"{layer_name}_{param_type}.csv"
        df.to_csv(os.path.join(directory, filename), header=None, index=False)

    print(f"All weights and biases have been saved to the '{directory}' directory.")

save_weights_to_csv(model, 'models/weights/3l/92/')

All weights and biases have been saved to the 'models/weights/3l/92/' directory.


In [29]:
def unpack_to_bits(dataset):
    dataset.map(lambda x: ";".join(np.char.mod('%d', np.unpackbits(np.array([x]).view(np.uint8))))).to_csv("dataset_bits.csv", sep=";", index=False)
    bity = pd.read_csv("dataset_bits.csv", dtype="uint64", sep=";", header=None)
    dataset.rename(columns={"draw": 768})
    pd.concat([bity, dataset.rename(columns={"draw": 768})], axis=1).to_csv("dataset_bits.csv", sep=";", index=False)